In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import csv


# Census & gmaps API Keys
from config import api_key_census
c = Census(api_key_census, year=2017)

filetomerge = "raleigh_zipcodes.csv"


#  Configure gmaps
#gmaps.configure(api_key=gkey)

In [2]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'zip code tabulation area:*'})


# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", 
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,Zipcode,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,00601,17599.0,11757.0,7041.0,11282.0,64.105915,13.943974
1,00602,39209.0,16190.0,8978.0,20428.0,52.100283,6.473004
2,00603,50135.0,16645.0,10897.0,25176.0,50.216416,7.156677
3,00606,6304.0,13387.0,5960.0,4092.0,64.911168,3.236041
4,00610,27590.0,18741.0,9266.0,12553.0,45.498369,5.342515


In [3]:
zipcodes_df = pd.read_csv(filetomerge)
#zipcodes_df.head()

In [4]:
# still working on merging two dataframes

raleigh_data = census_pd.merge(zipcodes_df, how='outer', on='Zipcode')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat